In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, regexp_replace, sum, when, sum as spark_sum, avg, rank, desc
from pymongo import MongoClient
from pyspark.sql.window import Window
import logging

In [2]:
# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
# Initialize SparkSession
spark = SparkSession.builder \
    .appName("Most popular product categories among different age groups") \
    .config("spark.executor.memory", "1g") \
    .getOrCreate()

In [4]:
# Define the HDFS directories
customer_data_dir = "hdfs://namenode:9000/csv_files/customers/"



In [5]:

# Create a static DataFrame to infer the schema for customer data
try:
    customer_static_df = spark.read \
        .format("csv") \
        .option("header", "true") \
        .load(customer_data_dir)
    customer_static_df.printSchema()
except Exception as e:
    logger.error(f"Error reading static data: {e}")
    spark.stop()
    raise

root
 |-- customer_id: string (nullable = true)
 |-- age: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- Item_Purchased: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Purchase_Amount_USD: string (nullable = true)
 |-- location: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Season: string (nullable = true)
 |-- Review Rating: string (nullable = true)
 |-- Subscription Status: string (nullable = true)
 |-- Shipping Type: string (nullable = true)
 |-- Discount Applied: string (nullable = true)
 |-- Promo Code Used: string (nullable = true)
 |-- Previous Purchases: string (nullable = true)
 |-- Payment Method: string (nullable = true)
 |-- Frequency of Purchases: string (nullable = true)



In [6]:
# Create a streaming DataFrame for customer data
try:
    customer_streaming_df = spark.readStream \
        .schema(customer_static_df.schema) \
        .option("header", "true") \
        .csv(customer_data_dir)
except Exception as e:
    logger.error(f"Error creating streaming DataFrame: {e}")
    spark.stop()
    raise

In [7]:
# MongoDB connection setup
try:
    #client = MongoClient(mongo_host, mongo_port)
    client = MongoClient("mongodb://mongodb:27017")
    db = client['Products']
    coll_high_revenue = db['high_revenue']
except Exception as e:
    logger.error(f"Error connecting to MongoDB: {e}")
    spark.stop()
    raise

In [8]:



def process_highest_sales_revenue(batch_df, batch_id):
   
   try:
         # Group by Item_Purchased and calculate total sales revenue
        product_sales = batch_df.groupBy("Item_Purchased").agg(
             spark_sum("Purchase_Amount_USD").alias("TotalSalesRevenue")
        )
      
    # Sort the results by TotalSalesRevenue in descending order
        sorted_product_sales = product_sales.orderBy(desc("TotalSalesRevenue"))
    
    # Convert the DataFrame to a list of dictionaries
        sorted_product_sales_list = [row.asDict() for row in sorted_product_sales.collect()]
    
    # Insert into MongoDB (assuming you have established a connection)
    # Replace 'db.highest_sales_revenue_products' with your MongoDB collection
    # db.highest_sales_revenue_products.insert_many(sorted_product_sales_list)
        
         # Insert into MongoDB if the list is not empty
        if sorted_product_sales_list:
          coll_high_revenue.insert_many(sorted_product_sales_list)
          sorted_product_sales.show()
          logger.info(f"Batch {batch_id} processed and inserted into MongoDB")
        else:
          logger.info(f"Batch {batch_id} processed but no high-value customers found")
    # Insert into MongoDB (assuming you have established a connection)
    # Replace 'db.high_value_customer_habits' with your MongoDB collection
  
   except Exception as e:
        logger.error(f"Error processing batch {batch_id}: {e}")
              

In [ ]:
# Write the streaming DataFrame to the console using the function
try:
   # Write the streaming DataFrame to the console using the function
   query_highest_sales_revenue = customer_streaming_df.writeStream \
    .outputMode("append") \
    .foreachBatch(process_highest_sales_revenue) \
    .start()
    # Await termination
   query_highest_sales_revenue.awaitTermination()
except Exception as e:
    logger.error(f"Error in streaming query: {e}")
finally:
    # Stop Spark session
    spark.stop()


INFO:py4j.java_gateway:Callback Server Starting
INFO:py4j.java_gateway:Socket listening on ('127.0.0.1', 37125)
INFO:py4j.clientserver:Python Server ready to receive messages
INFO:py4j.clientserver:Received command c on object id p0
ERROR:__main__:Error processing batch 0: name 'popular_categories_list' is not defined
